## Прогнозирование стоимости автомобиля по характеристикам
# Парсинг сайта "AUTO.RU"

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import ast

import os
for dirname, _, filenames in os.walk('./input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./input/BMW.csv
./input/NISSAN.csv
./input/INFINITI.csv
./input/my_urls.csv
./input/HONDA.csv
./input/test.csv
./input/AUDI.csv
./input/MITSUBISHI.csv
./input/VOLKSWAGEN.csv
./input/TOYOTA.csv
./input/VOLVO.csv
./input/LEXUS.csv
./input/all_auto_ru_09_09_2020.csv
./input/SKODA.csv
./input/MERCEDES.csv
./input/sample_submission.csv


Для начала посмотрим тестовый датасет

In [9]:
DIR_TEST = './input/'
DIR_OUT = './out/'
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [3]:
test.sample(5)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
2644,седан,AUDI,https://auto.ru/cars/used/sale/audi/a6/1101240...,чёрный,NaN,Внимание! Только для клиентов AVILON Автомобил...,1.8 LTR,190 N12,"{""cruise-control"":true,""esp"":true,""airbag-driv...",бензин,...,SEDAN ROBOT 1.8,роботизированная,EUROPEAN,2 владельца,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1721,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/rapid/110...,синий,"{""id"":""21158688"",""name"":""Ambition"",""available_...",Причина продажи — нужен а/м повыше. Комплект з...,1.6 LTR,110 N12,"{""cruise-control"":true,""esp"":true,""usb"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,3 года и 1 месяц,Оригинал,передний,Левый,Не требует ремонта,Растаможен
26072,седан,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/polo...,серый,NaN,- Один собственник\n- Оригинал ПТС\n- Автомоби...,1.6 LTR,105 N12,"{""electro-window-back"":true,""tinted-glass"":tru...",бензин,...,SEDAN AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
18071,внедорожник 5 дв.,MERCEDES,https://auto.ru/cars/used/sale/mercedes/gla_cl...,серый,NaN,Автомобиль с полностью прозрачной историей. Об...,2.0 LTR,211 N12,NaN,бензин,...,ALLROAD_5_DOORS ROBOT 2.0,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
2424,купе,AUDI,https://auto.ru/cars/used/sale/audi/a5/1101291...,белый,NaN,Выгода от 30 000 руб. при обмене на Ваш автомо...,2.0 LTR,225 N12,"{""cruise-control"":true,""engine-proof"":true,""as...",бензин,...,COUPE ROBOT 2.0,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [5]:
test.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня'],
      dtype='object')

Видим 32 столбца, из них только 6 - числовые. Т.е. у нас преимущественно категориальный датасет.

Для подготовки к парсингу создадим словарь brand_dict, где ключи - марки автомобилей, а значения - их кол-ва в test:

In [6]:
brands = test.brand.value_counts()
brand_dict = {brands.index[i]: brands[i] for i in range(len(brands))}
brand_dict

{'BMW': 4473,
 'VOLKSWAGEN': 4404,
 'NISSAN': 4393,
 'MERCEDES': 4180,
 'TOYOTA': 3913,
 'AUDI': 3421,
 'MITSUBISHI': 2843,
 'SKODA': 2741,
 'VOLVO': 1463,
 'HONDA': 1150,
 'INFINITI': 871,
 'LEXUS': 834}

## Сделаем парсинг сайта auto.ru следующим образом:
берем фильтр по Москве: легковые автомобили с пробегом 
берем фильтр по марке: автомобиля из словаря brand_dict, другие марки не трогаем 

### Первая часть парсинга данных:
Со всех страниц собираем URL - ссылки на объявления по каждому автомобилю, записываем сылки в отдельный DataFrame для каждой марки,
сохраняем эти DataFrame в отдельные файлы CSV

In [ ]:
cars_urls = {}
df_b = pd.DataFrame(columns=['url', 'brand'])
url = 'https://auto.ru/moskva/cars/'

# Проходим по каждому брэнду, выберем количество страниц = количество позиций для брэнда в тестовом датасете (избыточно)
for car_brand, amount in brand_dict.items():
    brand_url = url + car_brand + '/used/?page='
    cars_urls[car_brand] = []
    print('\n' + car_brand)
    for number in range(1, amount + 1):
        if (number % 10) == 0:
            print(number, end='\t')
        response = requests.get(brand_url+str(number),
                                headers={'User-Agent': 'Mozilla/5.0'})
        if response.status_code != 200:
            continue
        # Собираем все URL- ссылки на объявления на странице
        page = BeautifulSoup(response.text, 'html.parser')
        link_list = page.find_all(
            'a', class_='Link ListingItemTitle-module__link')
        if len(link_list) != 0:
            for link in link_list:
                cars_urls[car_brand].append(link['href'])
        else:
            break
    # Сохраняем все страницы в DataFrame, сохраняем DataFrame в файл.
    df = pd.DataFrame(cars_urls[car_brand], columns=['url'])
    df['brand'] = car_brand
    df_b = pd.concat([df_b, df], ignore_index=True)
    df.to_csv(car_brand + '.csv')

df_b.to_csv('my_urls.csv')
print('\nEND!')

In [7]:
for dirname, _, filenames in os.walk('./input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./input/BMW.csv
./input/NISSAN.csv
./input/INFINITI.csv
./input/my_urls.csv
./input/HONDA.csv
./input/test.csv
./input/AUDI.csv
./input/MITSUBISHI.csv
./input/VOLKSWAGEN.csv
./input/TOYOTA.csv
./input/VOLVO.csv
./input/LEXUS.csv
./input/all_auto_ru_09_09_2020.csv
./input/SKODA.csv
./input/MERCEDES.csv
./input/sample_submission.csv


### Получилось 12 файлов CSV (по названию брэндов), в каждом находятся ссылки на объявления о продаже.

### Вторая часть парсинга данных:

- Выбираем брэнд из списка
- считываем соответствующий файл с URL-ссылками на объявления в датафрейм
- проходим по датафрейму, выбираем по очереди URL, открываем страницу с соответствующим объявлением
- собираем всю необходимую информацию на странице, записываем в словарь
- добавляем словарь как строку в наш новый датафрейм с результатами
- сохраняем датафрейм с результатами для каждого бренда в отдельный CSV файл


In [ ]:
# Парсинг списка на странице
def parse_list(p_list, value):
    try:
        ss = p_list[p_list.index(value) + 1].replace('\xa0', ' ')
    except ValueError:
        ss = ''
    finally:
        return ss


# Парсинг страницы по ее URL, возвращаем словарь полученных значений
def parse_url(url):
    p_dict = {}
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    response.encoding = 'utf-8'
    page = BeautifulSoup(response.text, 'html.parser')
    if response.status_code != 200:
        print('!', end='\t')
        return p_dict

    page_test = page.find('script', type='application/ld+json')
    start, end = '<script type="application/ld+json">', '</script>'
    pg = str(page_test)[len(start):-len(end)]
    pg = pg.replace('\n', '\\n')
    tmp_dict = ast.literal_eval(pg)
    try:
        p_dict['bodyType'] = tmp_dict['bodyType']
        p_dict['brand'] = tmp_dict['brand']
        p_dict['car_url'] = url
        p_dict['color'] = tmp_dict['color']
        p_dict['engineDisplacement'] = tmp_dict['vehicleEngine']['engineDisplacement']
        p_dict['enginePower'] = tmp_dict['vehicleEngine']['enginePower']
        p_dict['fuelType'] = tmp_dict['vehicleEngine']['fuelType']
        p_dict['modelDate'] = tmp_dict['modelDate']
        p_dict['numberOfDoors'] = tmp_dict['numberOfDoors']
        p_dict['priceCurrency'] = tmp_dict['offers']['priceCurrency']
        p_dict['productionDate'] = tmp_dict['productionDate']
        p_dict['vehicleConfiguration'] = tmp_dict['vehicleConfiguration']
        p_dict['vehicleTransmission'] = tmp_dict['vehicleTransmission']
    except KeyError:
        p_dict['bodyType'] = ''

    link_list = page.find_all('div', title="Идентификатор объявления")
    try:
        p_dict['sell_id'] = link_list[0].text[2:]
    except IndexError:
        p_dict['sell_id'] = ''

    try:
        p_dict['price'] = tmp_dict['offers']['price']
    except KeyError:
        p_dict['price'] = ''

    try:
        link_list = page.find_all('script')
        a_string = str(link_list[10])
        ind = a_string.find('"model_info":{"code":')
        b_string = a_string[ind + 21:ind + 60].split(',')
        p_dict['model_name'] = b_string[0].replace('"', '')
    except IndexError:
        p_dict['model_name'] = ''

    try:
        link_list = page.find_all('span', class_='CardInfoRow__cell')
        link_list_a = []
        for i in link_list:
            link_list_a.append(i.text)
        p_dict['mileage'] = parse_list(link_list_a, 'Пробег').replace(' ', '').replace('км', '')
        p_dict['Владельцы'] = parse_list(link_list_a, 'Владельцы')
        p_dict['Владение'] = parse_list(link_list_a, 'Владение')
        p_dict['ПТС'] = parse_list(link_list_a, 'ПТС')
        p_dict['Привод'] = parse_list(link_list_a, 'Привод')
        p_dict['Руль'] = parse_list(link_list_a, 'Руль')
        p_dict['Состояние'] = parse_list(link_list_a, 'Состояние')
        p_dict['Таможня'] = parse_list(link_list_a, 'Таможня')
    except IndexError:
        p_dict['mileage'] = ''
    return p_dict


# Парсинг автомобильного брэнда по его названию, передаем в функцию название брэнда и количество разбираемых объявлений
# записываем результат в датафрейм, датафрейм сохраняем в файл CSV
def parse_brand(car_brand, num_iter):
    print('\n' + car_brand)
    df_col = pd.DataFrame(columns=['bodyType', 'brand', 'car_url', 'color',
                                   'engineDisplacement', 'enginePower',
                                   'fuelType', 'mileage', 'modelDate', 'model_name',
                                   'numberOfDoors', 'priceCurrency',
                                   'productionDate', 'sell_id', 'vehicleConfiguration',
                                   'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС',
                                   'Привод', 'Руль', 'Состояние', 'Таможня', 'price'])

    df_brand = pd.read_csv(DIR_TEST + car_brand + '.csv', index_col=0)
    max_len = len(df_brand) - 1
    for number in range(num_iter):
        if number > max_len:
            break
        url = df_brand.iloc[number, 0]
        df_col = df_col.append(parse_url(url), ignore_index=True)
        if (number % 100) == 0:
            print(number, end='\t')

    df_col.to_csv(DIR_OUT + car_brand + '_out.csv')
    print('\nEND for ' + car_brand)


# Парсинг всех брендов
# iter_multiplier - мультипликатор количества записей для бренда из тестового датасета
def parse_all(iter_multiplier):
    test = pd.read_csv(DIR_TEST + 'test.csv')
    brand_dict = {brands.index[i]: brands[i] for i in range(len(brands))}
    for car_brand, amount in brand_dict.items():
        parse_brand(car_brand, int(amount * iter_multiplier))
    print('\nEND for all brands!')


# Парсинг (с удвоенным количеством записей для каждого бренда из тестового датасета)
parse_all(2)

## Получили в результате 12 файлов CSV для для каждого бренда

In [11]:
for dirname, _, filenames in os.walk('./out'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./out/VOLKSWAGEN_out.csv
./out/BMW_out.csv
./out/AUDI_out.csv
./out/SKODA_out.csv
./out/LEXUS_out.csv
./out/MITSUBISHI_out.csv
./out/NISSAN_out.csv
./out/HONDA_out.csv
./out/VOLVO_out.csv
./out/TOYOTA_out.csv
./out/INFINITI_out.csv
./out/MERCEDES_out.csv


### Соберем все файлы в один датасет и выгрузим его в файл.

In [16]:
df_full = pd.DataFrame(columns=['bodyType', 'brand', 'car_url', 'color',
                                   'engineDisplacement', 'enginePower',
                                   'fuelType', 'mileage', 'modelDate', 'model_name',
                                   'numberOfDoors', 'priceCurrency',
                                   'productionDate', 'sell_id', 'vehicleConfiguration',
                                   'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС',
                                   'Привод', 'Руль', 'Состояние', 'Таможня', 'price'])
for car_brand in brand_dict.keys():
    df_temp = pd.read_csv(DIR_OUT + car_brand +'_out.csv')
    df_full = pd.concat([df_full, df_temp], ignore_index=True)
df_full.to_csv(DIR_OUT + 'full_out.csv')

# Данные готовы, можно проводить их обработку и построение моделей на их основе. 

In [18]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47954 entries, 0 to 47953
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              47923 non-null  object 
 1   brand                 47927 non-null  object 
 2   car_url               47927 non-null  object 
 3   color                 47927 non-null  object 
 4   engineDisplacement    47927 non-null  object 
 5   enginePower           47927 non-null  object 
 6   fuelType              47923 non-null  object 
 7   mileage               47927 non-null  float64
 8   modelDate             47923 non-null  float64
 9   model_name            47941 non-null  object 
 10  numberOfDoors         47923 non-null  float64
 11  priceCurrency         47923 non-null  object 
 12  productionDate        47923 non-null  float64
 13  sell_id               47927 non-null  float64
 14  vehicleConfiguration  47923 non-null  object 
 15  vehicleTransmission

In [20]:
df_full.sample(10)

,bodyType,brand,car_url,color,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,...,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price,Unnamed: 0
39674,седан,AUDI,https://auto.ru/cars/used/sale/audi/80/1102450...,серый,1.8 LTR,90 N12,бензин,250000.0,1986.0,80,...,механическая,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,100000.0,2346.0
604,кабриолет,BMW,https://auto.ru/cars/used/sale/bmw/1er/1101469...,белый,3.0 LTR,306 N12,бензин,52000.0,2007.0,1ER,...,автоматическая,3 или более,NaN,Дубликат,задний,Левый,Не требует ремонта,Растаможен,1930000.0,604.0
3063,седан,BMW,https://auto.ru/cars/used/sale/bmw/5er/1102973...,чёрный,2.0 LTR,190 N12,дизель,138000.0,2016.0,5ER,...,автоматическая,1 владелец,2 года и 9 месяцев,Оригинал,задний,Левый,Не требует ремонта,Растаможен,2390000.0,3063.0
39026,седан,AUDI,https://auto.ru/cars/used/sale/audi/a4/1103047...,коричневый,1.8 LTR,170 N12,бензин,85000.0,2011.0,A4,...,вариатор,2 владельца,1 год и 7 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1200000.0,1698.0
19469,седан,NISSAN,https://auto.ru/cars/used/sale/nissan/primera/...,красный,1.6 LTR,90 N12,бензин,235000.0,1995.0,PRIMERA,...,механическая,3 или более,1 год и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,85000.0,1715.0
20916,внедорожник 5 дв.,NISSAN,https://auto.ru/cars/used/sale/nissan/pathfind...,серый,2.5 LTR,190 N12,дизель,178000.0,2010.0,PATHFINDER,...,автоматическая,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1100000.0,3162.0
26021,хэтчбек 5 дв.,MERCEDES,https://auto.ru/cars/used/sale/mercedes/b_klas...,белый,1.6 LTR,122 N12,бензин,54000.0,2011.0,B_KLASSE,...,роботизированная,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,929000.0,4879.0
28079,минивэн,MERCEDES,https://auto.ru/cars/used/sale/mercedes/v_klas...,чёрный,2.1 LTR,190 N12,дизель,27.0,2014.0,V_KLASSE,...,автоматическая,1 владелец,1 год,Оригинал,полный,Левый,Не требует ремонта,Растаможен,19800000.0,6937.0
10783,седан,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/jett...,белый,1.6 LTR,110 N12,бензин,107000.0,2014.0,JETTA,...,механическая,2 владельца,4 месяца,Оригинал,передний,Левый,Не требует ремонта,Растаможен,770000.0,1837.0
44206,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,серебристый,1.8 LTR,180 N12,бензин,115000.0,2013.0,OCTAVIA,...,роботизированная,3 или более,2 года и 8 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен,735000.0,2170.0
